# Lab 7c: Web Search Tool - Live Satellite Visualization

## Why Web Search Matters: The Satellite Data Problem

🚀 **The space industry is exploding.** In 2023 alone, there were **212 orbital launches** worldwide. SpaceX's Starlink constellation grows by **~50 satellites per launch**, with launches happening almost weekly. 

This creates a challenge for AI models:
- **Training data cutoff**: Models are trained on data from months or years ago
- **Rapid change**: Satellite counts can increase by hundreds in a single month
- **Outdated answers**: Ask a model "how many Starlink satellites are there?" and you'll get stale data

**Solution: Web Search Tool** - Give your agent real-time access to current information!

---

In this lab, you'll use the **Web Search Preview Tool** to fetch real-time satellite data, then have an AI agent generate an interactive 3D visualization with accurate, up-to-date numbers.

> ⚠️ **Important**: The Web Search tool requires a **local model deployment** in the AI Foundry project. 
> Unlike other labs that can use only the APIM gateway, web search needs direct model access 
> for its internal processing. This lab deploys a local `gpt-4.1-mini` model alongside the APIM connection.

## What is Web Search Tool?

**Web Search Preview Tool** is a built-in tool in Foundry Agent Service that gives your agents access to real-time information from the public web. This enables:

| Capability | Use Case |
|------------|----------|
| **Real-time data** | Current satellite counts, launch statistics |
| **Fact verification** | Ground responses in current facts |
| **Research** | Gather information from multiple sources |
| **Up-to-date answers** | Overcome model knowledge cutoff |
| **Inline citations** | Automatic source attribution |

## Today's Mission

1. 🔍 **Search**: Use web search to find current satellite statistics
2. 🎨 **Generate**: Have the agent create an HTML visualization
3. 🌍 **Display**: Show animated satellites orbiting Earth in 3D

## Prerequisites
- Completed **Lab 1a** (Landing Zone with APIM)
- `.env` file with credentials

## Step 1: Install Dependencies

In [ ]:
!pip install azure-ai-projects==2.0.0b2 azure-ai-agents azure-identity -q

## Step 2: Configure Environment

In [ ]:
import subprocess
import os
import json
import time
import re
from pathlib import Path
from IPython.display import display, Markdown, HTML

# Load .env from parent directory
env_path = Path("../../.env")
if env_path.exists():
    for line in env_path.read_text().splitlines():
        if '=' in line and not line.startswith('#'):
            key, value = line.split('=', 1)
            os.environ[key.strip()] = value.strip()

# Landing Zone settings (from Lab 1a)
APIM_URL = os.environ.get("APIM_URL", "")
APIM_KEY = os.environ.get("APIM_KEY", "")
MODEL_NAME = os.environ.get("MODEL_NAME", "gpt-4.1-mini")

# Resource group for this lab
RG = "bing-grounding-lab"
LOCATION = "eastus2"

# Get current user info
PRINCIPAL_ID = subprocess.run(
    'az ad signed-in-user show --query id -o tsv',
    shell=True, capture_output=True, text=True
).stdout.strip()

SUBSCRIPTION_ID = subprocess.run(
    'az account show --query id -o tsv',
    shell=True, capture_output=True, text=True
).stdout.strip()

if not APIM_URL or not APIM_KEY:
    print("❌ Missing APIM_URL or APIM_KEY in .env file!")
    print("   Please complete Lab 1a first")
else:
    display(Markdown(f'''
### ✅ Configuration Loaded

| Setting | Value |
|---------|-------|
| APIM Gateway | `{APIM_URL[:50]}...` |
| Model | `{MODEL_NAME}` |
| Resource Group | `{RG}` |
'''))

## Step 3: Create Resource Group

⏱️ ~1 minute

In [ ]:
# Create resource group
!az group create -n "{RG}" -l "{LOCATION}" -o table

## Step 4: Deploy AI Foundry Project with Web Search

This deploys:
- **AI Foundry Account** with local chat model (required for web search)
- **APIM connection** for agent invocation

> 💡 No separate Bing resource needed - Web Search is a native capability in Foundry Agent Service!

⏱️ ~3-5 minutes

In [ ]:
!az deployment group create -g "{RG}" --template-file spoke.bicep \
    -p deployerPrincipalId="{PRINCIPAL_ID}" \
    -p apimUrl="{APIM_URL}" \
    -p apimSubscriptionKey="{APIM_KEY}" \
    -p gatewayModelName="{MODEL_NAME}" \
    -o table

In [ ]:
# Get deployment outputs
outputs = json.loads(subprocess.run(
    f'az deployment group show -g "{RG}" -n spoke --query properties.outputs -o json',
    shell=True, capture_output=True, text=True
).stdout)

PROJECT_ENDPOINT = outputs['projectEndpoint']['value']
APIM_CONNECTION = outputs['apimConnectionName']['value']
LOCAL_MODEL = outputs['localChatModel']['value']
GATEWAY_MODEL = f"{APIM_CONNECTION}/{outputs['gatewayModelName']['value']}"

display(Markdown(f'''
### ✅ Deployment Complete!

| Resource | Value |
|----------|-------|
| Project Endpoint | `{PROJECT_ENDPOINT[:50]}...` |
| Gateway Model | `{GATEWAY_MODEL}` |
| Local Model | `{LOCAL_MODEL}` (required for web search) |
'''))

## Step 5: Wait for RBAC Propagation

In [6]:
from IPython.display import clear_output

for i in range(60, 0, -10):
    clear_output(wait=True)
    print(f"⏳ Waiting for RBAC to propagate... {i}s")
    time.sleep(10)

clear_output(wait=True)
print("✅ RBAC permissions ready!")

✅ RBAC permissions ready!


---

# Mission: Satellite Visualization with Web Search

Now we'll create an agent that:
1. Uses **Web Search** to find current satellite statistics
2. Generates an **HTML visualization** with the data
3. Shows **animated satellites** orbiting Earth

In [8]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, WebSearchPreviewTool
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
project_client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)

# Get the OpenAI-compatible client for conversations/responses
openai_client = project_client.get_openai_client()

# Create agent with Web Search Preview Tool
agent = project_client.agents.create_version(
    agent_name="SatelliteResearcher",
    definition=PromptAgentDefinition(
        model=LOCAL_MODEL,  # Web search requires local model
        instructions="""You are a space data researcher and visualization specialist.

When asked about satellites:
1. Use web search to find the LATEST data about satellites currently in orbit
2. Look for statistics like:
   - Total number of operational satellites
   - Breakdown by operator (SpaceX Starlink, OneWeb, etc.)
   - Breakdown by orbit type (LEO, MEO, GEO)
   - Recent launches or changes
3. Cite your sources with URLs

When asked to create a visualization:
1. Generate a COMPLETE, self-contained HTML file
2. Show Earth in the center with animated orbiting dots
3. Include a stats panel showing the actual numbers

Output the complete HTML code in a ```html code block.""",
        tools=[WebSearchPreviewTool()],
    ),
    description="Agent for satellite research with web search.",
)

print(f"✅ Created SatelliteResearcher agent (name: {agent.name}, version: {agent.version})")

✅ Created SatelliteResearcher agent (name: SatelliteResearcher, version: 1)


In [46]:
# Step 1: Research current satellite data using Web Search
conversation = openai_client.conversations.create()
print(f"🧵 Conversation Created: {conversation.id}")

research_request = """
Search for the current number of satellites in Earth orbit as of 2026.

I need:
1. Total number of operational satellites
2. Number of Starlink satellites (SpaceX)
3. Number of OneWeb satellites
4. Breakdown by orbit type (LEO, MEO, GEO) if available
5. Any other major constellation operators

Please cite your sources with URLs.
"""

print("🔍 Searching the web for satellite data...")

response = openai_client.responses.create(
    conversation=conversation.id,
    input=research_request,
    extra_body={
        "agent": {
            "name": agent.name,
            "version": agent.version,
            "type": "agent_reference"
        }
    },
)

research_response = response.output_text
print(f"✅ Research complete!")
display(Markdown(f"### 🛰️ Satellite Research Results\n\n{research_response}"))

🧵 Conversation Created: conv_c91d16bba489901100nlGNNeuenp4kat0EkVhTiPLi149skb8S
🔍 Searching the web for satellite data...
✅ Research complete!


### 🛰️ Satellite Research Results

Here is the latest information on satellites in Earth orbit as of 2026:

1. Total number of operational satellites:
- Around 11,700 active satellites were in orbit as of mid-2025, and the number continues to increase rapidly, expected to exceed 12,000 by early 2026 [LiveScience](https://www.livescience.com/space/space-exploration/how-many-satellites-could-fit-in-earth-orbit-and-how-many-do-we-really-need).

2. Number of Starlink satellites (SpaceX):
- SpaceX Starlink has over 9,000 active satellites in orbit as of late 2025, with more than 10,600 launched in total.
- By early 2026, operational Starlink satellites number about 5,630 to 9,000 depending on the source, indicating continued launches and partial deorbiting.
- Starlink is the largest constellation, representing roughly 60% of all satellites in orbit [Tesla North](https://teslanorth.com/2025/12/13/starlink-set-to-cross-10000-satellites-by-early-2026), [LegalUnitedStates](https://legalunitedstates.com/how-many-starlink-satellites-are-in-orbit/).

3. Number of OneWeb satellites:
- Eutelsat ordered an additional 340 OneWeb LEO satellites in January 2026, adding to previously procured 100 satellites, making the current total around 440 satellites under contract.
- OneWeb is a significant but smaller constellation compared to Starlink [Airbus](https://www.airbus.com/en/newsroom/press-releases/2026-01-airbus-awarded-eutelsat-contract-for-further-340-low-earth-orbit).

4. Breakdown by orbit type:
- About 8,135 satellites are active in Low Earth Orbit (LEO).
- Approximately 552 satellites operate in Geostationary Orbit (GEO).
- Around 200 satellites are in Medium Earth Orbit (MEO).
- Other classifications include 19 satellites in high Earth orbit or graveyard orbits [Sci-Tech Today](https://www.sci-tech-today.com/stats/satellite-launch-statistics/).

5. Other major constellation operators:
- Amazon's Project Kuiper and China's Guowang constellation are notable emerging megaconstellations.
- Several countries and companies continue to contribute to the rapidly growing LEO satellite population.

Summary:
- Total operational satellites: ~11,700+ (growing to around 12,000+)
- Starlink active satellites: 5,600 to 9,000+ (largest constellation by far)
- OneWeb satellites: ~440 under contract currently
- Orbit breakdown: LEO (~8,135), GEO (~552), MEO (~200)
- Other major players: Amazon Kuiper, China’s Guowang, among others

Sources:
- https://teslanorth.com/2025/12/13/starlink-set-to-cross-10000-satellites-by-early-2026/
- https://www.livescience.com/space/space-exploration/how-many-satellites-could-fit-in-earth-orbit-and-how-many-do-we-really-need
- https://www.airbus.com/en/newsroom/press-releases/2026-01-airbus-awarded-eutelsat-contract-for-further-340-low-earth-orbit
- https://www.sci-tech-today.com/stats/satellite-launch-statistics/
- https://legalunitedstates.com/how-many-starlink-satellites-are-in-orbit/

In [87]:
# Step 2: Generate HTML visualization based on the research
viz_request = """
Based on the satellite data you found, create an HTML simulation using WebGL.

Use this minimal WebGL template and update the satellite count:

```html
<!DOCTYPE html>
<html>
<head>
    <title>Satellites</title>
    <style>
        *{margin:0;padding:0}
        body{background:#0a0a0f;overflow:hidden}
        canvas{display:block}
        .info{position:fixed;bottom:20px;left:20px;color:#444;font:12px monospace}
        .info span{color:#888}
    </style>
</head>
<body>
<div class="info"><span id="n">0</span> satellites in orbit</div>
<canvas id="c"></canvas>
<script>
const TOTAL = 8500; // UPDATE THIS from research
const DISPLAY = 800;

const c = document.getElementById('c');
const gl = c.getContext('webgl');
c.width = innerWidth; c.height = innerHeight;
gl.viewport(0,0,c.width,c.height);

const vs = `
attribute vec3 p;
uniform mat4 m;
void main(){
    gl_Position = m * vec4(p,1.0);
    gl_PointSize = 1.5;
}`;
const fs = `
precision lowp float;
void main(){
    float d = length(gl_PointCoord - 0.5);
    if(d > 0.5) discard;
    gl_FragColor = vec4(0.7, 0.75, 0.8, 1.0 - d*1.5);
}`;

function sh(t,s){const x=gl.createShader(t);gl.shaderSource(x,s);gl.compileShader(x);return x;}
const pg = gl.createProgram();
gl.attachShader(pg, sh(gl.VERTEX_SHADER, vs));
gl.attachShader(pg, sh(gl.FRAGMENT_SHADER, fs));
gl.linkProgram(pg); gl.useProgram(pg);

const pLoc = gl.getAttribLocation(pg,'p');
const mLoc = gl.getUniformLocation(pg,'m');

// Earth
const earth = [];
for(let i=0;i<3000;i++){
    const u = Math.random()*Math.PI*2, v = Math.acos(2*Math.random()-1);
    earth.push(Math.sin(v)*Math.cos(u), Math.cos(v), Math.sin(v)*Math.sin(u));
}
const eBuf = gl.createBuffer();
gl.bindBuffer(gl.ARRAY_BUFFER, eBuf);
gl.bufferData(gl.ARRAY_BUFFER, new Float32Array(earth), gl.STATIC_DRAW);

// Satellites
const sats = [];
for(let i=0;i<DISPLAY;i++){
    const r = 1.1 + Math.random()*0.8;
    const inc = (Math.random()*0.7+0.1)*Math.PI;
    const raan = Math.random()*Math.PI*2;
    const ph = Math.random()*Math.PI*2;
    const sp = 0.002 + Math.random()*0.008;
    sats.push({r,inc,raan,ph,sp});
}
const satPos = new Float32Array(DISPLAY*3);
const sBuf = gl.createBuffer();

function perspective(f,a,n,far){
    const t=1/Math.tan(f/2);
    return new Float32Array([t/a,0,0,0,0,t,0,0,0,0,(far+n)/(n-far),-1,0,0,2*far*n/(n-far),0]);
}
function lookAt(e,c,u){
    const z=norm([e[0]-c[0],e[1]-c[1],e[2]-c[2]]);
    const x=norm(cross(u,z));
    const y=cross(z,x);
    return new Float32Array([x[0],y[0],z[0],0,x[1],y[1],z[1],0,x[2],y[2],z[2],0,-dot(x,e),-dot(y,e),-dot(z,e),1]);
}
function mult(a,b){
    const r=new Float32Array(16);
    for(let i=0;i<4;i++)for(let j=0;j<4;j++){r[i*4+j]=0;for(let k=0;k<4;k++)r[i*4+j]+=b[i*4+k]*a[k*4+j];}
    return r;
}
function norm(v){const l=Math.sqrt(v[0]*v[0]+v[1]*v[1]+v[2]*v[2]);return[v[0]/l,v[1]/l,v[2]/l];}
function cross(a,b){return[a[1]*b[2]-a[2]*b[1],a[2]*b[0]-a[0]*b[2],a[0]*b[1]-a[1]*b[0]];}
function dot(a,b){return a[0]*b[0]+a[1]*b[1]+a[2]*b[2];}

gl.enable(gl.BLEND);
gl.blendFunc(gl.SRC_ALPHA,gl.ONE);

let t=0;
function draw(){
    t+=0.01;
    gl.clearColor(0.04,0.04,0.06,1);
    gl.clear(gl.COLOR_BUFFER_BIT);
    
    const proj = perspective(0.8, c.width/c.height, 0.1, 100);
    const view = lookAt([Math.sin(t*0.2)*8,2,Math.cos(t*0.2)*8],[0,0,0],[0,1,0]);
    const mvp = mult(proj,view);
    gl.uniformMatrix4fv(mLoc, false, mvp);
    
    // Draw earth
    gl.bindBuffer(gl.ARRAY_BUFFER, eBuf);
    gl.enableVertexAttribArray(pLoc);
    gl.vertexAttribPointer(pLoc,3,gl.FLOAT,false,0,0);
    gl.drawArrays(gl.POINTS,0,earth.length/3);
    
    // Update & draw satellites
    for(let i=0;i<DISPLAY;i++){
        const s=sats[i];
        s.ph+=s.sp;
        let x=s.r*Math.cos(s.ph), y=0, z=s.r*Math.sin(s.ph);
        const ci=Math.cos(s.inc),si=Math.sin(s.inc);
        const y1=y*ci-z*si, z1=y*si+z*ci;
        const cr=Math.cos(s.raan),sr=Math.sin(s.raan);
        satPos[i*3]=x*cr+z1*sr; satPos[i*3+1]=y1; satPos[i*3+2]=-x*sr+z1*cr;
    }
    gl.bindBuffer(gl.ARRAY_BUFFER, sBuf);
    gl.bufferData(gl.ARRAY_BUFFER, satPos, gl.DYNAMIC_DRAW);
    gl.vertexAttribPointer(pLoc,3,gl.FLOAT,false,0,0);
    gl.drawArrays(gl.POINTS,0,DISPLAY);
    
    requestAnimationFrame(draw);
}

document.getElementById('n').textContent = TOTAL.toLocaleString();
onresize=()=>{c.width=innerWidth;c.height=innerHeight;gl.viewport(0,0,c.width,c.height);};
draw();
</script>
</body>
</html>
```

Update the `TOTAL` variable with the actual satellite count from your research.
Output the complete HTML in a ```html code block.
"""

print("🎨 Generating HTML visualization...")

viz_response = openai_client.responses.create(
    conversation=conversation.id,
    input=viz_request,
    extra_body={
        "agent": {
            "name": agent.name,
            "version": agent.version,
            "type": "agent_reference"
        }
    },
)

print(f"✅ Visualization generated!")

🎨 Generating HTML visualization...
✅ Visualization generated!


In [88]:
# Extract HTML and save it
text = viz_response.output_text
html_content = None

# Extract HTML from code block
html_match = re.search(r'```html\s*([\s\S]*?)```', text)
if html_match:
    html_content = html_match.group(1).strip()
    print("✅ Extracted HTML visualization!")
else:
    # Try to find HTML without code block markers
    if '<!DOCTYPE html>' in text or '<html' in text:
        start = text.find('<!DOCTYPE html>') if '<!DOCTYPE html>' in text else text.find('<html')
        end = text.rfind('</html>') + 7
        if start >= 0 and end > start:
            html_content = text[start:end]
            print("✅ Extracted HTML visualization!")

# Save HTML file
if html_content:
    html_path = "satellite_visualization.html"
    with open(html_path, "w") as f:
        f.write(html_content)
    print(f"\n💾 Saved visualization to: {html_path}")
    print(f"   File size: {len(html_content):,} bytes")
else:
    print("⚠️ Could not extract HTML from response")
    print(text[:2000])

✅ Extracted HTML visualization!

💾 Saved visualization to: satellite_visualization.html
   File size: 4,298 bytes


In [92]:
# Display the visualization inline in the notebook
import base64

if html_content:
    # Use base64 encoding to preserve all JavaScript properly
    b64 = base64.b64encode(html_content.encode()).decode()
    display(HTML(f'''
    <div style="border-radius: 8px; overflow: hidden; margin: 20px 0;">
        <iframe src="data:text/html;base64,{b64}" width="100%" height="500px" style="border: none;"></iframe>
    </div>
    '''))
else:
    print("No visualization to display")

---

# 🧹 Cleanup

In [40]:
# Cleanup agent
print("🧹 Cleaning up...")

try:
    project_client.agents.delete_version(agent.name, agent.version)
    print(f"   Deleted {agent.name} agent (v{agent.version})")
except Exception as e:
    print(f"   Note: {e}")

print("✅ Cleanup complete!")

🧹 Cleaning up...
   Deleted SatelliteResearcher agent (v1)
✅ Cleanup complete!


In [41]:
# Optional: Delete resource group (includes Bing Search resource)
# Uncomment to delete all resources
# !az group delete -n "{RG}" --yes --no-wait
# print("🗑️ Resource group deletion initiated")

---

# 🎉 Summary

## What You Built

| Step | Tool | What Happened |
|------|------|---------------|
| 1. Research | **Web Search Preview** | Searched for current satellite data |
| 2. Generate | **LLM** | Created HTML visualization code |
| 3. Display | **Notebook** | Rendered animated satellite view |

## Key Learnings

1. **Web Search Preview Tool** gives agents access to real-time web data
2. No separate Bing resource needed - it's built into Foundry Agent Service
3. Agents can **generate code** (HTML, CSS, JS) based on live data
4. **Combine tools** for powerful workflows (search → generate → display)
5. Always **cite sources** when using web-grounded data

## Web Search Use Cases

| Use Case | Example |
|----------|--------|
| **Market research** | Current stock prices, competitor info |
| **News summaries** | Latest headlines on a topic |
| **Fact checking** | Verify claims with current sources |
| **Data visualization** | Charts from live data (like this lab!) |
| **Technical docs** | Find latest API documentation |

## Enterprise Considerations

| Consideration | Recommendation |
|---------------|----------------|
| **Data freshness** | Web search returns real-time results |
| **Source quality** | Consider filtering to trusted domains |
| **Privacy** | Data sent to Grounding with Bing (see terms) |
| **Cost** | Web search incurs usage costs |
| **Admin control** | Can be disabled at subscription level |

## Web Search Modes

| Mode | Description | Best For |
|------|-------------|----------|
| **Non-reasoning** | Direct query → top sources | Quick lookups |
| **Reasoning** | Multi-step planning (gpt-5) | Complex research |
| **Deep Research** | Extended investigation | Legal/scientific research |

---

*Data grounded in reality, visualizations limited only by imagination.* 🛰️